In [1]:
from typing import List
from openptv_python.tracking_frame_buf import Frame
from openptv_python.parameters import VolumePar, ControlPar
from openptv_python.calibration import Calibration
from openptv_python.correspondences import Correspond, find_candidate
from openptv_python.epi import Coord2d, epi_mm
# from openptv_python.tracking_frame_buf import Target
MAXCAND = 1000

In [2]:

def match_pairs(corr_list: List[List[List[Correspond]]], corrected: List[List[Coord2d]], frm: Frame, vpar: VolumePar, cpar: ControlPar, calib: List[Calibration]):
    """
    Match pairs of cameras.

    This function matches pairs of cameras by finding corresponding points in each camera. The correspondences are stored in the `corr_lists` argument.

    The following steps are performed:
    1. For each pair of cameras, the epipolar lines for the two cameras are calculated.
    2. For each target in the first camera, the corresponding points in the second camera are found by searching along the epipolar line.
    3. The correspondences are stored in the `corr_lists` argument.

    The `corr_lists` argument is a list of lists of lists of `Correspond` objects. Each inner list corresponds to a pair of cameras, and each inner-most list corresponds to a correspondence between two points in the two cameras. The `Correspond` objects have the following attributes:
    * `p1`: The index of the target in the first camera.
    * `p2`: The index of the target in the second camera.
    * `corr`: The correspondence score.
    * `dist`: The distance between the two points.

    The following are the arguments for the function:
    * `corr_lists`: A list of lists of lists of `Correspond` objects. Each inner list corresponds to a pair of cameras, and each inner-most list corresponds to a correspondence between two points in the two cameras.
    * `corrected`: A list of lists of `Coord2d` objects. Each inner list corresponds to a camera, and each inner-most object corresponds to the corrected coordinates of a target in that camera.
    * `frm`: A `frame` object.
    * `vpar`: A `VolumePar` object.
    * `cpar`: A `ControlPar` object.
    * `calib`: A list of `Calibration` objects.

    The function returns None.
    """
    for i1 in range(cpar.num_cams - 1):
        for i2 in range(i1 + 1, cpar.num_cams):
            for i in range(frm.num_targets[i1]):
                xa12, ya12, xb12, yb12 = epi_mm(corrected[i1][i].x, corrected[i1][i].y, calib[i1], calib[i2], cpar.mm, vpar)
                # origin point in the corr_list
                corr_list[i1][i2][i].p1 = i
                pt1 = corrected[i1][i].pnr
                # search for a conjugate point in corrected[i2]
                cand = find_candidate(corrected[i2], frm.targets[i2], frm.num_targets[i2], xa12, ya12, xb12, yb12, frm.targets[i1][pt1].n, frm.targets[i1][pt1].nx, frm.targets[i1][pt1].ny, frm.targets[i1][pt1].sumg, vpar, cpar, calib[i2])
                # write all corresponding candidates to the preliminary corr_list of correspondences
                count = len(cand)
                if count > MAXCAND:
                    count = MAXCAND
                for j in range(count):
                    corr_list[i1][i2][i].p2[j] = cand[j].pnr
                    corr_list[i1][i2][i].corr[j] = cand[j].corr
                    corr_list[i1][i2][i].dist[j] = cand[j].tol
                    corr_list[i1][i2][i].n = count


In [3]:

# def test_match_pairs():
# Test case 1
corr_list = [[[Correspond() for _ in range(10)] for _ in range(10)] for _ in range(10)]
corrected = [[Coord2d() for _ in range(10)] for _ in range(10)]
frm = Frame(num_cams=2)
vpar = VolumePar()
cpar = ControlPar()
calib = [Calibration() for _ in range(10)]
match_pairs(corr_list, corrected, frm, vpar, cpar, calib)


In [4]:
corr_list[0][0][0].p2

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [5]:

assert corr_list[0][1][0].p1 == 0
assert corr_list[0][1][0].p2[0] == 1
assert corr_list[0][1][0].corr[0] == 0.5
assert corr_list[0][1][0].dist[0] == 1.0
assert corr_list[0][1][0].n == 1


AssertionError: 

In [ ]:

# Test case 2
corr_list = [[[Correspond() for _ in range(5)] for _ in range(5)] for _ in range(5)]
corrected = [[Coord2d() for _ in range(5)] for _ in range(5)]
frm = Frame(4)
vpar = VolumePar()
cpar = ControlPar()
calib = [Calibration() for _ in range(5)]
match_pairs(corr_list, corrected, frm, vpar, cpar, calib)
assert corr_list[2][3][1].p1 == 1
assert corr_list[2][3][1].p2[0] == 2
assert corr_list[2][3][1].corr[0] == 0.8
assert corr_list[2][3][1].dist[0] == 1.2
assert corr_list[2][3][1].n == 1
